# Examen Final

- Prender cámaras.
- Puede utilizar sus apuntes.
- Puede utilizar los notebooks del curso.
- La alumna o alumno que sea sorprendido solicitando ayuda a cualquier persona, sea ella compañera o no, será calificada con un 2.
- Cada pregunta vale un punto.
- La nota mínima es un 2.
- La nota se determinará con la siguiente fórmula $nota = \min\left(puntos + 2, 7\right)$
- Todas las funciones necesarias para resolver los ejercicios están ya importadas al notebook.

## Librerías

In [1]:
from finrisk import examen_final as ef
from scipy.interpolate import interp1d
import modules.hull_white as hw
import pandas as pd
import numpy as np
import textwrap
import random
import math

## Curva Cero Cupón

Los plazos están en días y las tasas en convención exp act/365.

In [2]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [3]:
df_curva.head()

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929


In [4]:
curva = interp1d(df_curva['plazo'], df_curva['tasa'], 'linear', fill_value='extrapolate')

## Cálculo Opción

Para identificar si es una Call o una Put se usa un `enum`.

In [5]:
c_p = hw.CallPut.CALL
c_p = hw.CallPut.PUT

Ver toda la documentación:

In [13]:
print(hw.zcb_call_put.__doc__)


    Calcula el valor de una call o una put sobre un bono cero cupón en el modelo de HW.
    
    params:
    
    - c_p: indica si es la opción es Call o Put. Es un `enum` de tipo CallPut. Ejemplo, c_p = CallPut.CALL.
    - strike: es el strike de la opción. Se ingresa como número. Un strike del 90% se ingresa como .9.
    - r0: es la tasa corta al momento de valorizar la opción (t = 0).
    - to: instante de tiempo en que vence la opción, expresado en años.
    - tb: instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
    - zo: factor de descuento de mercado a tiempo t = 0 hasta to.
    - zb: factor de descuento de mercado a tiempo t = 0 hasta tb.
    - gamma: parámetro gamma del modelo HW.
    - sigma: parámetro sigma del modelo HW.
    
    return:
    
    -  el valor de la opción.
    


## Valores `gamma` y `sigma`

In [7]:
gamma = 1
sigma = .005

In [8]:
strikes = [.98, .981, .982, .983, .984, .985, .99, .991, .992, .993]
for k in strikes:
    print(k, hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        curva(.0001),
        1,
        2,
        math.exp(-curva(365)),
        math.exp(-curva(730) * 730 / 365.0),
        gamma,
        sigma
    ))

0.98 0.019553575799963863
0.981 0.0185542779311344
0.982 0.017554980062305048
0.983 0.016555682193475696
0.984 0.015556384324646233
0.985 0.014557086455816881
0.99 0.0095605971116699
0.991 0.008561299242840548
0.992 0.007562001374011196
0.993 0.006562703505181733


## Preguntas

In [9]:
try:
    q = ef.get_questions()
except Exception as e:
    print(str(e))

In [10]:
for qq in q:
    print(f'{qq[0]}\n')
    print(textwrap.fill(f'{qq[1]}', 80))
    print('\n')

Pregunta 1:

Considere un swap a 1Y con cupones semestrales de 1.7% (lineal). Si la tasa
cupón cero a 0.5Y es 1.6% (exp), calcule la tasa cero a 1Y.


Pregunta 2:

Usando la curva cero cupón entregada y parámetros gamma = 1.0 y sigma = 0.5%
valorice una call a 1Y sobre un bono cero cupón a 2Y con un strike de 99.0%.
Entregue el resultado 8 decimales y suponiendo un nocional de 1.


Pregunta 3:

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a -.7028 .


Pregunta 4:

Explique la diferencia entre valorizar un payoff g(r(T), T) por simulación de
Montecarlo utilizando el modelo de Hull-White en la medida libre de riesgo y el
modelo de Hull-White en la medida T-forward.


Pregunta 5:

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,214.00 (365 días después).


Pregunta 6:

Una opción sobre un bono cupón cero valorizada con el modelo de Vasicek 

## Respuestas

### Pregunta 1

Considere un swap a 1Y con cupones semestrales de 1.7% (lineal). Si la tasa
cupón cero a 0.5Y es 1.6% (exp), calcule la tasa cero a 1Y.

In [38]:
#1.6/100=math.exp(r*0.5)
r_exp=math.log(1.6/100)/0.5
tasa_cero=math.exp(r_exp*1) #es el factor no la tasa 
print(f'Valor de la tasa cero exponencial a 1Y: {tasa_cero:,.8%}') 

Valor de la tasa cero a 1Y: 0.02560000%


### Pregunta 2

Se pide valorizar una call. 

In [39]:
#Datos importantes: 
k=0.99
r0=curva(.0001) #es la tasa corta al momento de valorizar la opción (t = 0).
to=1 #instante de tiempo en que vence la opción, expresado en años call
tb=2 # instante de tiempo en que vence el bono subyacente, expresado en años. Debe ser tb > to.
zo= math.exp(-curva(365)) # a un año factor de descuento de mercado a tiempo t = 0 hasta to.
zb= math.exp(-curva(730) * 730 / 365.0) #factor de descuento de mercado a tiempo t = 0 hasta tb.

In [40]:
CALL=hw.zcb_call_put(
        hw.CallPut.CALL,
        k,
        r0,
        to,
        tb,
        zo,
        zb,
        gamma,
        sigma
    )
print(f'Valor de la opción Call: {CALL:,.8f}') #considerando nocional de 1

Valor de la opción Call: 0.00956060


### Pregunta 3

Calcule un paso de simulación de Montecarlo para el modelo de Vasicek usando
gamma = 1.0, sigma = 0.5%, dt = 1/264, r0 obtenido de la curva cupón cero
entregada y número aleatorio N(0, 1) igual a -.7028 .

In [41]:
gamma = 1.0
sigma = 0.005
r_ = 0.005 #dicho en prueba 
dt = 1/264
rnd_al= -.7028
r0 = curva(0)
#Se usará la misma idea de la función del notebook 12, 12.2 
dt_gamma_r_ = dt * gamma * r_
sigma_sqdt = sigma * math.sqrt(dt)
result = [(0, r0),]
gamma_dt = gamma * dt   
r = dt_gamma_r_ + (1 - gamma_dt) * r0 + sigma_sqdt * rnd_al
print(f'Un paso de la simulación de MC para la tas del modelo de Vasicek:{r:.8%}')

Un paso de la simulación de MC para la tas del modelo de Vasicek:0.06058245%


### Pregunta 4

La primera diferencia que se encuentra en la valorización por simulación de Monte Carlo con la media forward versus la medida libre de riesgo es el drift. En la medida forward el drift cambia, para calcularlo se debe derivar la función del bono cupón cero de Hull & White y al cambiar el drift se tendrá una nueva función de $\theta$ que es la que va a generar una diferencia en la simulación de Monte-Carlo. En el resto de las cosas, con esto me refiero a la creación de los valores normales aleatorios, las funciones involucradas como A y B. Recordar que la función $\theta$ SÍ cambia. 

Una vez que se tiene eso se puede usar la misma función para simular pero cambaindo la función $\theta$ y se crean los ciclos for para que recorra las diferentes trayectorias (número de simulaciones) y los pasos que se desean entregar. 

La segunda diferencia que ocurre es a la hora de traer a valor presente la opción, con la medida forward se tiene que el la esperanza del factor de descuento es conocida y se considera como el factor de descuento con la tasa al vencimiento, por lo que para traer a valor presente basta multiplicar ese factro de descuento con el promedio de los payoffs obtenidos. Cabe mencionar que en este caso el número de pasos serñia 264. 

Por otro lado, si se hace la simulación con la medida libre de riesgo, se tiene que la cantidad de pasos que hace la simulación son 263, y para poder traer a valor presente se debe calcular el factor de descuento de cada trayectoria (simulación) y multplicarlo por el payoff de cada trayectoria y a la multiplicación anterior se le calcula el promedio. 



### Pregunta 5

La TNA viene dada por la siguiente ecuación: 
$$TNA=Round\left(\left( \frac{ICP_{T}}{ICP_{t}}-1\right)\cdot\frac{360}{d},4\right)$$

Donde:

- TNA: significa *tasa nominal anual*.
- $ICP_{\tau}$: es el valor del $ICP$ la fecha $\tau=t,T$.
- $d$ es el número de días entre $t$ y $T$.
- $Round\left(x,4\right)$ redondea el número $x$ a 4 decimales.

Calcule la TNA con ICP0 = 10,000.00 e ICP365 = 10,214.00 (365 días después).

In [42]:
ICP0=10000.00
ICP=10214.00
d=365
TNA=round((ICP/ICP0-1)*360/d,4)
print(f'Valor de la TNA: {TNA:,.4f}')

Valor de la TNA: 0.0211


### Pregunta 6

Una opción sobre un bono cupón cero valorizada con el modelo de Vasicek vale lo
mismo que la misma opción valorizada con el modelo de Hull-White. Comente.

Es importante comenzar mencionando que el modelo de HUll-White es una extensión del modelo de Vasicek. El modelo de Vacisek no logra percibir los cambios de curvatura en la curva cero cupón mientras que el del Hull-White si puede, es a partir de lo anterior que la primera diferencia entre estos dos modelos, es la dinámica que sigue la tasa instantánea, ya que en el modelo de HW aparece una función $\theta$ que se puede interpretar como un nivel de reversión a la media variable en el tiempo, mientras que en el otro modelo no está. 

A partir de lo anterior, la fórmula para valorizar el bono cupón cero cambia, sin embargo, si se quiere valorizar una opción europea (Call o Put) la fórmula sigue siendo la misma, por lo que en ambos modelos el valor de la opción es el mismo (si es que es una call o put). En otro caso, en otro tipo de opción, no creo que la opción en ambos modelos valga lo mismo, ya que, como se mencionó anteriormente la medida de riesgo varía y como por lo general no se tiene una fórmula cerrada para otro tipo de opciones y se debe valorizar por MC, el valor no sería el mismo. 


### Pregunta 7

Para calcular la tasa forward se debe en primer lugar interpolar la curva. 

Una vez que se tiene la curva, se deben establecer los datos necesarios. 

In [43]:
t1=282
t2=454 
r1=curva(t1)
r2=curva(t2)
df1=math.exp(-r1* t1/365)
df2=math.exp(-r2* t2/365)

Luego se calcular la tasa forward, como la tasa equivalente. 

In [44]:
tasa_forward=(df1/df2-1)*360/(t2-t1)
print(f'Valor de la tasa forward: {tasa_forward:,.8%}')

Valor de la tasa forward: 0.05902083%
